Import Library. Selain pandas dan numpy, library baru yang diperlukan untuk topic modelling adalah gensim dan spacy. Jika coba di jupiter notebook atau local machine install dengan pip install --upgrade gensim

In [1]:
import pandas as pd
import numpy as np

import re
import string

import spacy
#!pip install --upgrade gensim
import gensim
from gensim import corpora

Library yang lain yang diperlukan adalah pyldavis. Jika run di jupiter notebook atau mesin lain install dengan pip install pyldavis. Jika belum terinstall, install terlebih dahulu

In [2]:
# libraries for visualization
!pip install pyldavis


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 4.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=4fedd8e989c52df451fbb92e9470f0b03f412c4a9e278fb3e14365585fd21361
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=26b12eb3da310931302b1944f650b03c9a01097f6c4945850374f9f5256fa4ed
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built pyldavis sklearn


Kita import library nya

In [3]:
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


Kita loading filenya di google drive (Reviews_Filter.csv)

In [4]:
from google.colab import drive
drive.mount('/content/drive')
review_data = pd.read_csv('./drive/MyDrive/Colab Notebooks/Reviews_Filter.csv')
review_data.head()
#print(review_data.head(2))
#print(len(review_data))
#print('Unique Products')
#print(len(review_data.groupby('ProductId')))
#print('Unique Users')
#print(len(review_data.groupby('UserId')))

Mounted at /content/drive


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


Fungsi untuk membersihkan data: Membuang punctuation, membuat huruf kecil

In [5]:
def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>3))]) 
    
    return text2.lower()

import nltk dan import library untuk stopword (kata sambung, article)

In [6]:
import nltk
nltk.download('stopwords') # run this one time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Mambuang index dan menjadikan Id sebagai index

In [7]:
review_data.dropna(axis = 0, how ='any',inplace=True) 

In [ ]:
review_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


Aplikasikan fungsi 'clean_text' yang sudah didefinisikan diatas dan menambahkan kolom baru bernama 'num_word_text' yang berisi jumlah kata

In [8]:
review_data['Text'] = review_data['Text'].apply(clean_text)
review_data['Num_words_text'] = review_data['Text'].apply(lambda x:len(str(x).split())) 


Kita intip untuk kita bandingkan dengan sebelumnya

In [9]:
review_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Num_words_text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,have bought several vitality canned food produ...,30
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,product arrived labeled jumbo salted peanutsth...,20
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",this confection that been around centuries lig...,52
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,looking secret ingredient robitussin believe h...,22
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,great taffy great price there wide assortment ...,17


Memfilter review yang jumlah katanya antara 20-100. Hasilnya disimpan di variabel bernama df_short_reviews

In [10]:
max_review_data_sentence_length  = review_data['Num_words_text'].max()

mask = (review_data['Num_words_text'] < 100) & (review_data['Num_words_text'] >=20)
df_short_reviews = review_data[mask]

In [11]:
df_short_reviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Num_words_text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,have bought several vitality canned food produ...,30
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,product arrived labeled jumbo salted peanutsth...,20
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",this confection that been around centuries lig...,52
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,looking secret ingredient robitussin believe h...,22
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,wild hair taffy ordered this five pound taffy ...,43


Bandingkan sebelum dan sesudah di Filter

In [12]:
len(review_data.index)

14999

In [13]:
len(df_short_reviews.index)

9921

Cleaning tambahan yaitu stopwords

In [14]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
df_short_reviews['Text']=df_short_reviews['Text'].apply(remove_stopwords)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [15]:
df_short_reviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Num_words_text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,bought several vitality canned food products f...,30
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,product arrived labeled jumbo salted peanutsth...,20
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",confection around centuries light pillowy citr...,52
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,looking secret ingredient robitussin believe f...,22
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,wild hair taffy ordered five pound taffy enjoy...,43


lemmatization = membuat jadi kata dasar. Dan memfilter hanya menggunakan noun (kata benda) juga adjective (kata sifat). Yang dibuang adalah verb (kata kerja). Lalu dimasukan ke kamus (dictionary)

In [16]:
#nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])
import en_core_web_sm
nlp = en_core_web_sm.load()
def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value [] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],


In [17]:
text_list=df_short_reviews['Text'].tolist()
print(text_list[1])
tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[1])

product arrived labeled jumbo salted peanutsthe peanuts actually small sized unsalted sure error vendor intended represent product jumbo
['product', 'jumbo', 'peanutsthe', 'peanut', 'small', 'sized', 'unsalted', 'sure', 'error', 'vendor', 'product', 'jumbo']


In [18]:
dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

Initialisi model. LDA -> teknik untuk topik modelling
Kita harus memberi input berapa jumlah topik yang kita mau. Sekarang kita coba jumlah topiknya ada 7

In [19]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=7, random_state=100,
                chunksize=1000, passes=50,iterations=250)

Output streaming akan dipotong hingga 5000 baris terakhir.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np

Print topik2 nya dan kata2 nya yang terkait

In [20]:
lda_model.print_topics()

[(0,
  '0.028*"product" + 0.028*"good" + 0.024*"great" + 0.020*"price" + 0.020*"store" + 0.017*"amazon" + 0.014*"time" + 0.013*"chip" + 0.013*"flavor" + 0.010*"order"'),
 (1,
  '0.025*"taste" + 0.023*"good" + 0.022*"sugar" + 0.018*"great" + 0.018*"flavor" + 0.017*"cracker" + 0.015*"water" + 0.014*"drink" + 0.013*"sweet" + 0.013*"product"'),
 (2,
  '0.117*"coffee" + 0.026*"good" + 0.024*"flavor" + 0.017*"taste" + 0.016*"kcup" + 0.016*"strong" + 0.015*"bean" + 0.013*"roast" + 0.011*"great" + 0.011*"vanilla"'),
 (3,
  '0.016*"candy" + 0.015*"cake" + 0.012*"free" + 0.010*"coconut" + 0.010*"pocket" + 0.009*"product" + 0.008*"flour" + 0.008*"brownie" + 0.008*"packaging" + 0.007*"chili"'),
 (4,
  '0.060*"treat" + 0.045*"food" + 0.037*"dog" + 0.022*"ingredient" + 0.015*"organic" + 0.012*"product" + 0.012*"natural" + 0.012*"soda" + 0.011*"small" + 0.011*"healthy"'),
 (5,
  '0.068*"chocolate" + 0.052*"cookie" + 0.023*"peanut" + 0.015*"butter" + 0.014*"milk" + 0.011*"good" + 0.010*"dark" + 0.007*

Visualkan dengan pylDavis

In [21]:
# Visualize the topics
#https://github.com/bmabey/pyLDAvis
#https://speakerdeck.com/bmabey/visualizing-topic-models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, doc_term_matrix, dictionary)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.141086 -0.002915       1        1  29.346417
1     -0.122624 -0.035459       2        1  17.321683
2     -0.142834 -0.137108       3        1  15.509216
6     -0.073364  0.032724       4        1  13.727854
4      0.018591  0.267886       5        1  10.530893
5      0.223553 -0.199020       6        1   7.006684
3      0.237765  0.073892       7        1   6.557254, topic_info=          Term         Freq        Total Category  logprob  loglift
201     coffee  3466.000000  3466.000000  Default  30.0000  30.0000
66       treat  1210.000000  1210.000000  Default  29.0000  29.0000
185  chocolate   907.000000   907.000000  Default  28.0000  28.0000
0         food  1693.000000  1693.000000  Default  27.0000  27.0000
570     cookie   690.000000   690.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
131       blue    44.319065   113.252249   Topic7  -5.6408   1.7864
376    problem    50.084971   460.259364   Topic7  -5.5185   0.5065
428       last    46.565030   373.892109   Topic7  -5.5914   0.6415
211      water    47.037504   748.104294   Topic7  -5.5813  -0.0420
362       easy    46.406265   506.216152   Topic7  -5.5948   0.3351

[419 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
169       1  0.775811       able
169       6  0.115219       able
169       7  0.107538       able
952       2  0.983228  afternoon
450       5  0.996750    allergy
...     ...       ...        ...
120       2  0.262046       year
120       3  0.011698       year
120       4  0.080720       year
120       5  0.155590       year
120       7  0.046794       year

[652 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 7, 5, 6, 4])

In [22]:
print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Output streaming akan dipotong hingga 5000 baris terakhir.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np


Perplexity:  -7.472004164832765

Coherence Score:  0.3501209713510807


In [23]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 3.8 MB/s 
     |████████████████████████████████| 636 kB 25.4 MB/s 
     |████████████████████████████████| 88 kB 7.3 MB/s 
     |████████████████████████████████| 5.2 MB 51.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 85 kB 5.0 MB/s 
     |████████████████████████████████| 5.5 MB 39.9 MB/s 
     |████████████████████████████████| 1.3 MB 18.4 MB/s 
     |████████████████████████████████| 163 kB 49.9 MB/s 
     |████████████████████████████████| 7.6 MB 42.3 MB/s 
     |████████████████████████████████| 1.1 MB 44.4 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp37-cp37m-linux_x86_64.whl size=2340729 sha256=cd7f3cb3be2603b9b322a9fe1bc07311cfa57bedab6556674fe66fb08a4c935e
  Stored in directory: /root/.cache

In [24]:
!pip install bertopic[visualization]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
#import packages

import pandas as pd 
import numpy as np
from bertopic import BERTopic

/usr/local/lib/python3.7/dist-packages/tornado/httputil.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  class HTTPHeaders(collections.MutableMapping):


In [26]:
#load data 
import pandas as pd 
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('./drive/MyDrive/Colab Notebooks/Reviews_Filter.csv')
 
# select only 6000 tweets 
df = df[0:6000]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
df.head(10)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [45]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
df_short_reviews['Text']=df_short_reviews['Text'].apply(remove_stopwords)

In [46]:
df_short_reviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Num_words_text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,bought several vitality canned food products f...,30
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,product arrived labeled jumbo salted peanutsth...,20
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",confection around centuries light pillowy citr...,52
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,looking secret ingredient robitussin believe f...,22
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,wild hair taffy ordered five pound taffy enjoy...,43


In [47]:
# create model 
 
model = BERTopic(verbose=True)
 
#convert to list 
docs = df.Text.to_list()
 
topics, probabilities = model.fit_transform(docs)

Batches:   0%|          | 0/188 [00:00<?, ?it/s]

2022-11-07 12:35:21,373 - BERTopic - Transformed documents to Embeddings
2022-11-07 12:35:38,642 - BERTopic - Reduced dimensionality
2022-11-07 12:35:38,901 - BERTopic - Clustered reduced embeddings


In [48]:
model.get_topic_freq().head(11)

,Topic,Count
0,-1,1462
1,0,392
2,1,205
3,2,121
4,3,110
5,4,110
6,5,108
7,6,107
8,7,106
9,8,105


In [49]:
model.get_topic(0)

[('dog', 0.03259286504434868),
 ('food', 0.025424886556315102),
 ('dogs', 0.02033126117290326),
 ('newmans', 0.013367127935975592),
 ('her', 0.012652565654862585),
 ('she', 0.012169857021623098),
 ('he', 0.011410493990887834),
 ('my', 0.010233399815118121),
 ('his', 0.010030833423879866),
 ('and', 0.008787923227562324)]

In [50]:
model.visualize_topics()

In [51]:
model.visualize_barchart()

In [52]:
model = BERTopic(nr_topics=30)

In [53]:
topics, probabilities = model.fit_transform(docs)

2022-11-07 13:00:55,559 - BERTopic - Transformed documents to Embeddings
2022-11-07 13:01:12,843 - BERTopic - Reduced dimensionality
2022-11-07 13:01:13,124 - BERTopic - Clustered reduced embeddings
2022-11-07 13:01:18,708 - BERTopic - Reduced number of topics from 107 to 31


In [54]:
model.visualize_barchart()

Analisis kali ini adalah kita bisa melihat bahwa ketika kita membatasi n_topic itu sangat berpengaruh dengan topik yang paling dominan muncul, di topik 0 yang paling dominan muncul adalah dog dan ketika topiknya sama un ternya di tpik 0 masih dog yang dominan muncul.

namun ketika melihat topik 1 yang belum dibatasi dengaan yang sudah dibatasi itu sangat berbeda sekali, kali di topic satu yang belum di batasi yang paling dominan adalah tea namu di topik 1 yang sudah dibatasi adalah chocolate. ini menandakan bahwa pembatasan topic sangat berpengaruh karena probabilitasnya semakin spesifik.

yang menjadi perbedaan yang sangat signifikan adalah di tpik 2 sebelum di batasi dan sesudah dibatasi. di tipik 2 yang sebelumnya yang sring muncuk itu adalah baby,mungkin maksudnya adalah makanan baby, tapi di topik yang 2 ketika sudah di batasi bertopiknya yang dominan muncul adalah minuman tea